In [10]:
import numpy as np
import pandas as pd
import pyspark.sql.functions as F
import regex as re

from IPython.display import display
from pyspark.sql import DataFrame, SparkSession
from pyspark.sql.types import *
from typing import *

In [2]:
spark = SparkSession \
    .builder \
    .appName('group2nba') \
    .getOrCreate()

In [22]:
path_main = '/project/ds5559/group2nba'

T = TypeVar('T')

In [13]:
@F.udf(IntegerType())
def get_has_possession(team: str, plays: List[str]) -> int:
    '''Get: whether the current team has possession of the ball'''
    return int(bool(team) and any([bool(x) for x in plays]))


@F.udf(IntegerType())
def get_score_diff(score1: int, score2: int) -> int:
    '''Get: score differential relative to the specified team'''
    return score1 - score2


@F.udf(IntegerType())
def get_secleft_total(quarter: int, maxquarter: int, sec: int) -> int:
    '''Get: SecLeft by Quarter, accounting for OT'''
    if quarter < 5:
        return ((maxquarter - 4) * 300) + ((4 - quarter) * 720) + sec
    else:
        return ((maxquarter - quarter) * 300) + sec
    
    
@F.udf(StringType())
def get_team(team: str) -> str:
    '''Get: the specified team'''
    return team
    
    
@F.udf(IntegerType())
def get_won(winner: str, team: str) -> int:
    '''Get: whether the specified team won'''
    return int(winner == team)
    
    
@F.udf(IntegerType())
def get_year(date: str) -> int:
    '''Get: Year of game took place in'''
    return int(re.match(r'[A-Z][a-z]+ \d+ (\d{4})', date).groups()[0])

In [20]:
# .withColumn('TimeElapsed', F.udf(lambda x: 2880 - x, IntegerType())('SecLeftTotal')) \
# .withColumn('SecExp', get_time_exp('MaxQuarter', 'SecLeftTotal')) \

def build_model_win_percent(df: DataFrame) -> DataFrame:
    '''Constucts a model for predicting the win percent on a play by play basis'''
    altered = df \
        .groupBy(['Date', 'HomeTeam', 'AwayTeam']) \
        .agg(F.max('Quarter')
             .alias('MaxQuarter')) \
        .join(df, ['Date', 'HomeTeam', 'AwayTeam']) \
        .withColumn('SecLeftTotal', get_secleft_total('Quarter', 'MaxQuarter', 'SecLeft')) \
        .withColumn('Year', get_year('Date'))

    home = altered \
        .withColumn('Won', get_won('WinningTeam', 'HomeTeam')) \
        .withColumn('ScoreDiff', get_score_diff('HomeScore', 'AwayScore')) \
        .withColumn('HasPossession', get_has_possession('HomePlay', F.array(*POSSESSION_PLAYS))) \
        .withColumn('Team', get_team('HomeTeam')) \
        .select(MODEL_FIELDS)
    
    away = altered \
        .withColumn('Won', get_won('WinningTeam', 'AwayTeam')) \
        .withColumn('ScoreDiff', get_score_diff('AwayScore', 'HomeScore')) \
        .withColumn('HasPossession', get_has_possession('AwayPlay', F.array(*POSSESSION_PLAYS))) \
        .withColumn('Team', get_team('AwayTeam')) \
        .select(MODEL_FIELDS)
    
    return home.union(away)

In [15]:
FIELDS = {
      'Url': StringType
    , 'GameType': StringType
    , 'Location': StringType
    , 'Date': StringType
    , 'Time': StringType
    , 'WinningTeam': StringType
    , 'Quarter': IntegerType
    , 'SecLeft': IntegerType
    , 'AwayTeam': StringType
    , 'AwayPlay': StringType
    , 'AwayScore': IntegerType
    , 'HomeTeam': StringType
    , 'HomePlay': StringType
    , 'HomeScore': IntegerType
    , 'Shooter': StringType
    , 'ShotType': StringType
    , 'ShotOutcome': IntegerType
    , 'ShotDist': IntegerType
    , 'Assister': StringType
    , 'Blocker': StringType
    , 'FoulType': StringType
    , 'Fouler': StringType
    , 'Fouled': StringType
    , 'Rebounder': StringType
    , 'ReboundType': IntegerType
    , 'ViolationPlayer': StringType
    , 'ViolationType': StringType
    , 'TimeoutTeam': StringType
    , 'FreeThrowShooter': StringType
    , 'FreeThrowOutcome': IntegerType
    , 'FreeThrowNum': StringType
    , 'EnterGame': StringType
    , 'LeaveGame': StringType
    , 'TurnoverPlayer': StringType
    , 'TurnoverType': StringType
    , 'TurnoverCause': StringType
    , 'TurnoverCauser': StringType
    , 'JumpballAwayPlayer': StringType
    , 'JumpballHomePlayer': StringType
    , 'JumpballPoss': StringType
}

POSSESSION_PLAYS = [
      'Shooter'
    , 'Assister'
    , 'Fouled'
    , 'Rebounder'
    , 'ViolationPlayer'
    , 'FreeThrowShooter'
    , 'TurnOverPlayer'
    , 'JumpballPoss'
]

MODEL_FIELDS = [
      'Date'
    , 'HomeTeam'
    , 'AwayTeam'
    , 'Team'
    , 'Year'
    , 'Won'
    , 'ScoreDiff'
    , 'Quarter'
    , 'SecLeftTotal'
    , 'HasPossession'
]

In [16]:
schema = StructType([StructField(k, v()) for k, v in FIELDS.items()])

df_train = spark.read \
    .format('csv') \
    .option('header', True) \
    .schema(schema) \
    .load(f'{path_main}/clean_train_data/*')

display(df_train.count())
display(df_train.printSchema())
display(df_train.head(2))

2403581

root
 |-- Url: string (nullable = true)
 |-- GameType: string (nullable = true)
 |-- Location: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Time: string (nullable = true)
 |-- WinningTeam: string (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- SecLeft: integer (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- AwayPlay: string (nullable = true)
 |-- AwayScore: integer (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- HomePlay: string (nullable = true)
 |-- HomeScore: integer (nullable = true)
 |-- Shooter: string (nullable = true)
 |-- ShotType: string (nullable = true)
 |-- ShotOutcome: integer (nullable = true)
 |-- ShotDist: integer (nullable = true)
 |-- Assister: string (nullable = true)
 |-- Blocker: string (nullable = true)
 |-- FoulType: string (nullable = true)
 |-- Fouler: string (nullable = true)
 |-- Fouled: string (nullable = true)
 |-- Rebounder: string (nullable = true)
 |-- ReboundType: integer (nullable = tru

None

[Row(Url='/boxscores/201810160BOS.html', GameType='regular', Location='TD Garden Boston Massachusetts', Date='October 16 2018', Time='8:00 PM', WinningTeam='BOS', Quarter=1, SecLeft=700, AwayTeam='PHI', AwayPlay='R. Covington misses 3-pt jump shot from 27 ft', AwayScore=0, HomeTeam='BOS', HomePlay=None, HomeScore=0, Shooter='R. Covington - covinro01', ShotType=None, ShotOutcome=0, ShotDist=None, Assister=None, Blocker=None, FoulType=None, Fouler=None, Fouled=None, Rebounder=None, ReboundType=None, ViolationPlayer=None, ViolationType=None, TimeoutTeam=None, FreeThrowShooter=None, FreeThrowOutcome=None, FreeThrowNum=None, EnterGame=None, LeaveGame=None, TurnoverPlayer=None, TurnoverType=None, TurnoverCause=None, TurnoverCauser=None, JumpballAwayPlayer=None, JumpballHomePlayer=None, JumpballPoss=None),
 Row(Url='/boxscores/201810160BOS.html', GameType='regular', Location='TD Garden Boston Massachusetts', Date='October 16 2018', Time='8:00 PM', WinningTeam='BOS', Quarter=1, SecLeft=700, Aw

In [21]:
df_train = build_model_win_percent(df_train)

display(df_train.count())
display(df_train.printSchema())
display(df_train.head(2))

4807162

root
 |-- Date: string (nullable = true)
 |-- HomeTeam: string (nullable = true)
 |-- AwayTeam: string (nullable = true)
 |-- Team: string (nullable = true)
 |-- Year: integer (nullable = true)
 |-- Won: integer (nullable = true)
 |-- ScoreDiff: integer (nullable = true)
 |-- Quarter: integer (nullable = true)
 |-- SecLeftTotal: integer (nullable = true)
 |-- HasPossession: integer (nullable = true)



None

[Row(Date='December 18 2016', HomeTeam='MEM', AwayTeam='UTA', Team='MEM', Year=2016, Won=0, ScoreDiff=0, Quarter=1, SecLeftTotal=2880, HasPossession=0),
 Row(Date='December 18 2016', HomeTeam='MEM', AwayTeam='UTA', Team='MEM', Year=2016, Won=0, ScoreDiff=-3, Quarter=1, SecLeftTotal=2862, HasPossession=0)]